In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50

In [3]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 4s 0us/step


In [4]:
# Normalize the dataset
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [5]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=x_train.shape[1:]))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [7]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [8]:
# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

Epoch 1/50
391/391 [==============================] - 8s 8ms/step - loss: 1.6466 - accuracy: 0.3942 - val_loss: 1.2626 - val_accuracy: 0.5478
Epoch 2/50
391/391 [==============================] - 2s 6ms/step - loss: 1.2396 - accuracy: 0.5556 - val_loss: 1.0603 - val_accuracy: 0.6241
Epoch 3/50
391/391 [==============================] - 2s 6ms/step - loss: 1.0668 - accuracy: 0.6204 - val_loss: 0.9349 - val_accuracy: 0.6664
Epoch 4/50
391/391 [==============================] - 2s 6ms/step - loss: 0.9527 - accuracy: 0.6641 - val_loss: 0.8681 - val_accuracy: 0.6921
Epoch 5/50
391/391 [==============================] - 2s 6ms/step - loss: 0.8817 - accuracy: 0.6906 - val_loss: 0.8218 - val_accuracy: 0.7115
Epoch 6/50
391/391 [==============================] - 2s 6ms/step - loss: 0.8184 - accuracy: 0.7116 - val_loss: 0.7850 - val_accuracy: 0.7274
Epoch 7/50
391/391 [==============================] - 2s 6ms/step - loss: 0.7636 - accuracy: 0.7314 - val_loss: 0.7346 - val_accuracy: 0.7459
Epoch 

In [9]:
# Save model and weights
model.save('cifar10_model.h5')

In [10]:
# Evaluate the performance of our trained model
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 1s 2ms/step - loss: 0.6708 - accuracy: 0.8048
Test loss: 0.6708400845527649
Test accuracy: 0.8047999739646912
